# Bird Species Prediction Using Deep Learning

#### BIRD behavior and population trends have become an important issue now a days. Birds help us to detect other organisms in the environment (e.g. insects they feed on) easily as they respond quickly to the environmental changes. But, gathering and collecting information about birds requires huge human effort as well as becomes a very costlier method. In such case, a reliable system that will provide large scale processing of information about birds and will serve as a valuable tool for researchers, governmental agencies, etc. is required. So, bird species identification plays an important role in identifying that a particular image of bird belongs to which species. Bird species identification means predicting the bird species belongs to which category by using an image. 

### About Dataset-   
#### Data set of 525 bird species. 84635 training images, 2625 test images(5 images per species) and 2625 validation images(5 images per species. This is a very high quality dataset where there is only one bird in each image and the bird typically takes up at least 50% of the pixels in the image. As a result even a moderately complex model will achieve training and test accuracies in the mid 90% range.         

## 1. Imports:

In [ ]:
#For Warnings- 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Basic imports-

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib, os, random

In [ ]:
#Imports for CNN-

import os
import keras
import tensorflow as tf

from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout , GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.callbacks import Callback, EarlyStopping,ModelCheckpoint

In [ ]:
#Checking the version used-

print('Keras Version Used:',keras.__version__)
print('Tensorflow Version Used:',tf.__version__)

## 2. EDA:

### i.) Reading and Understanding Data-

In [ ]:
#Create a dataframe from the csv-

bird = pd.read_csv('D:/Dataset/birds.csv')

In [ ]:
#Clean column names-

bird.columns = [col.replace(' ', '_').lower() for col in bird.columns]

In [ ]:
print(bird)

In [ ]:
#Checking first ten records-

bird.head()

In [ ]:
#Checking last ten records-

bird.tail()

In [ ]:
#Count number of rows and cols-

bird.shape

In [ ]:
#Count no of rows axis=0

bird.shape[0]

In [ ]:
#Count no of rows axis=1

bird.shape[1]

In [ ]:
#Columns name-

list(bird.columns)

In [ ]:
#Info about csv file data-

bird.info

In [ ]:
bird.value_counts("data_set").head()

In [ ]:
#The describe() method returns description of the data in the DataFrame -

bird.describe()

In [ ]:
#Checking the distribution of classes-
#Frequency of bird species in the whole dataset

counts = bird['class_id'].value_counts()
print(counts)

In [ ]:
#Look at csv entries for one single bird:

#mask = birds_df['labels'].str.contains("ABBOTTS BABBLER") # Search for text fragment
#mask = birds_df.query('labels == "ABBOTTS BABBLER"') # query for name (case sensitive!)

mask = bird.loc[bird['class_id'] == 0]
print(mask.value_counts("data_set"))
mask

In [ ]:
#Visualize the distribution of classes-

plt.figure(figsize = (100, 80))
counts.plot(kind = 'bar')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Distribution of Classes')
plt.show()

### ii.) Image Sizes-

In [ ]:
#Important Imports -

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2 as cv

In [ ]:
bird.filepaths[0]

In [ ]:
image_path = 'D:\\Dataset\\train\\ABBOTTS BABBLER\\001.jpg'
img = Image.open(image_path)
img

In [ ]:
img.size

### iii.) Duplicates -

In [ ]:
bird.duplicated().sum()

### iv.) Missing Values-

In [ ]:
bird.isnull().sum()

### v.) Kurtosis-

In [ ]:
bird.kurtosis()

### vi.) Skewness-

In [ ]:
bird.skew()

### vii.)  Visualization-

In [ ]:
#Knowing Dataset-

import plotly.express as px

def tuple_count(file_path, dataset):
    bird_count = []
    for file in os.listdir(file_path):
        bird_count.append((file, len(os.listdir(file_path + file)), dataset))
    return bird_count

consolidated = tuple_count(test_path, 'test') + tuple_count(valid_path, 'valid') + tuple_count(train_path, 'train')
count_df = pd.DataFrame.from_records(consolidated, columns =['Name', 'Count', 'From']) 

fig = px.bar(count_df, x='Name', y='Count', color='From')
fig.update_xaxes(visible=False)
fig.show()

In [ ]:
#Create a bar plot -

plt.figure(figsize=(20, 10))
plt.bar(counts.index, counts.values)
plt.xlabel('Bird Species')
plt.ylabel('Count')
plt.title('Distribution of Bird Species')
plt.show()

In [ ]:
#Importing Some Images-

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_path = 'D:\\dataset\\train\\ABBOTTS BOOBY\\002.jpg'
img = mpimg.imread(img_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
#Randomly select 9 images:

samples = np.random.choice(bird.index, size=9, replace=False)

#Create a 3x3 grid to display the images:

base_path = 'D:/' 
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(12, 10))
for i, idx in enumerate(samples):
    
    print(i)
    print(idx)
 
    img_path = base_path+bird.filepaths[i]
    img = Image.open(img_path)
    row = i // 3
    col = i % 3
    ax[row, col].imshow(img)
    ax[row, col].set_title(bird['filepaths'][idx])
    ax[row, col].axis('off')
    
plt.show()

In [ ]:
#Visualize Image size-

import numpy as np
widths = np.random.randint(100, 500, size=100) # generate 100 random integers between 100 and 500
heights = np.random.randint(100, 500, size=100) # generate 100 random integers between 100 and 500
print(widths)
print(heights)

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(widths, heights)
plt.xlabel('Width')
plt.ylabel('Height')
plt.title('Distribution of Image Sizes')
plt.show()

### viii.) Correlation-

In [ ]:
bird.corr()

## Making a listing of the species names, training, test, validation file counts and prinitng all species -

In [ ]:
def print_in_color(txt_msg,fore_tupple,back_tupple,):
    
    #prints the text_msg in the foreground color specified by fore_tupple with the background specified by back_tupple 
    #text_msg is the text, fore_tupple is foregroud color tupple (r,g,b), back_tupple is background tupple (r,g,b)
    
    rf,gf,bf=fore_tupple
    rb,gb,bb=back_tupple
    msg='{0}' + txt_msg
    mat='\33[38;2;' + str(rf) +';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' +str(gb) + ';' + str(bb) +'m' 
    print(msg .format(mat))

In [ ]:
sdir=r'D:\\dataset\\'
train_dir=os.path.join(sdir, 'train')
msg='{0:8s}{1:4s}{2:^28s}{1:4s}{3:8s}{1:3s}{4:8s}{1:3s}{5:7s}{6}'
msg=msg.format('Class Id', ' ', 'Bird Specie  ', 'Train ','Test', 'Valid ','\n')
print_in_color(msg, (0,255,255), (0,0,0))
species_list= sorted(os.listdir(train_dir))
for i, specie in enumerate (species_list):
    file_path=os.path.join(train_dir,specie)
    train_files_list=os.listdir(file_path)
    train_file_count=str(len(train_files_list))
    msg='{0:^8s}{1:4s}{2:^28s}{1:4s}{3:^8s}{1:1s}{4:^8s}{1:3s}{5:^8s}'
    msg=msg.format(str(i), ' ',specie, train_file_count,'5', '5')
    toggle=i% 2   
    if toggle==0:
        back_color=(255,255,255)
    else:
        back_color=(191, 239, 242)
    print_in_color(msg, (0,0,0), back_color)
#print('\33[0m')

In [ ]:
#Printing All Species-

test_dir = os.path.join(sdir, 'test')
test_species_list=sorted(os.listdir(test_dir))
classes = len(os.listdir(test_dir))
fig = plt.figure(figsize=(20,250))
if classes % 5==0:
    rows=int(classes/5)
else:
    rows=int(classes/5) +1    
for row in range(rows):
    for column in range(5):
        i= row * 5 + column         
        if i>classes-1:
            break            
        specie=test_species_list[i]
        species_path=os.path.join(test_dir, specie)
        f_path=os.path.join(species_path, '1.jpg')        
        img = mpimg.imread(f_path)
        a = fig.add_subplot(rows, 5, i+1)
        imgplot=plt.imshow(img)
        a.axis("off")
        a.set_title(specie)

## About Classes-

In [ ]:
train_path="D:\\dataset\\train\\"
test_path="D:\\dataset\\test\\"
valid_path="D:\\dataset\\valid\\"

In [ ]:
#No. of Classes-

no_birds_classes = os.listdir(train_path)
len(no_birds_classes)

In [ ]:
#Name of Classes-

data_dir = pathlib.Path("D:\\dataset\\train\\")
BirdClasses = np.array(sorted([item.name for item in data_dir.glob("*")]))
print(BirdClasses)

## Visualization of Images-

In [ ]:
#Reading Data-

train_dir = "D:\\dataset\\train\\"
val_dir = "D:\\dataset\\valid\\"
test_dir = "D:\\dataset\\test\\"

In [ ]:
def view_random_image(target_dir, target_class):
  
    # setting up the image directory
    target_folder = target_dir + target_class

    #get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

    #read image and plotting it
    img = plt.imread(target_folder + "/" + random_image[0] )
    
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off")

    #print(f"Image shape: {img.shape}")
  
    return img

####  A function called 'view_random_image' that takes a directory path (target_dir) and a class/category name (target_class) as input. It combines the directory path and class name to create a specific folder path, retrieves a random image from that folder, reads and loads the image, displays the image using Matplotlib, sets the title of the plot as the class name and removes the axis labels from the plot. Finally, it returns the loaded image.

In [ ]:
img = view_random_image(train_path,'AMERICAN FLAMINGO')

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

random_index = np.random.randint(0 , len(BirdClasses)-1 , 16)

for i, ax in enumerate(axes.flat):
    ax.imshow(view_random_image(train_path,BirdClasses[random_index[i]]))
    ax.set_title(BirdClasses[random_index[i]])

## Data Preprocessing-

### Using MobileNet Model

#### MobileNet-v2 is a convolutional neural network that is 53 layers deep. You can load a pretrained version of the network trained on more than a million images from the ImageNet database. The pretrained network can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals.

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory( train_dir , target_size=(224,224) , batch_size=32 ,
                                           class_mode = "categorical" ,shuffle=True )

val_data = val_gen.flow_from_directory( val_dir , target_size=(224,224) , batch_size=32 , 
                                       class_mode = "categorical" , shuffle=True )

test_data = test_gen.flow_from_directory( test_dir , target_size=(224,224) , batch_size=32 , 
                                         class_mode = "categorical" ,shuffle=False )

#### ImageDataGenerator is a class in TensorFlow (specifically in the tf.keras.preprocessing.image module) that provides a way to augment and preprocess image data during training. It is commonly used in deep learning tasks, particularly in computer vision, to generate batches of image data with real-time data augmentation.

## Preparing MobileNet Pretrained Model-

#### MobileNetV2 is a popular deep learning model for image classification. It is a lightweight and efficient model that was designed to run on mobile devices with limited computational resources.

In [ ]:
mobilenet = MobileNetV2( include_top=False , weights="imagenet" , input_shape=(224,224,3))

In [ ]:
mobilenet.summary()

## Fine-Tuning MobileNet Model-

In [ ]:
#Freezing all layers except the last 20 layers

mobilenet.trainable=True

for layer in mobilenet.layers[:-20]:
    layer.trainable=False

#### The above code is freezing all layers except the last 20 layers. Freezing a layer means that its parameters will not be updated during the training process. By freezing most of the layers and only keeping the last 20 layers trainable, you can focus the training process on refining the final layers to better fit the new task or dataset.

In [ ]:
Model = Sequential([mobilenet,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(256,activation='relu'),
    BatchNormalization(),
    Dense(525,activation='softmax')
])

Model.summary()

In [ ]:
Model.compile( optimizer="adam", loss="categorical_crossentropy" , metrics=["accuracy"] )

In [ ]:
# Create Callback Checkpoint-

#checkpoint_path = "BirdsSpecies_Model_Checkpoint"
#checkpoint_callback = ModelCheckpoint(checkpoint_path,monitor="val_accuracy",save_best_only=True)

callbacks = [EarlyStopping(monitor='val_accuracy' , patience=5 , restore_best_weights=True)]

#### A Callback is a set of functions that can be applied at various stages during the training process. Callbacks provide a way to customize and extend the behavior of the training process without modifying the training loop itself.

In [ ]:
history = Model.fit(train_data, epochs=2, batch_size=32,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Saving model for further loading and again running because epochs are taking time in this project-

Model.save('bird.h5')

In [ ]:
#Loading saved model-

Model.load_weights('bird.h5')

In [ ]:
history = Model.fit(train_data, epochs=3, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

#### In above code I used batch size 64 instead of 32 because it has several advantages like it trains faster, stable gradient estimate and improved generalization.

In [ ]:
#Again Saving Model-

Model.save('bird2.h5')

In [ ]:
#Again loading saved model-

Model.load_weights('bird2.h5')

In [ ]:
history = Model.fit(train_data, epochs=5, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Again Saving Model-

Model.save('bird3.h5')

In [ ]:
#Again loading saved model-

Model.load_weights('bird3.h5')

In [ ]:
history = Model.fit(train_data, epochs=2, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Again Saving Model-

Model.save('bird4.h5')

In [ ]:
#Again loading saved model-

Model.load_weights('bird4.h5')

In [ ]:
history = Model.fit(train_data, epochs=5, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Again Saving Model-

Model.save('bird6.h5')

In [ ]:
#Again loading saved model-

Model.load_weights('bird6.h5')

In [ ]:
history = Model.fit(train_data, epochs=3, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Again Saving Model-

Model.save('bird7.h5')

In [ ]:
#Loading Model-

Model.load_weights('bird7.h5')

In [ ]:
history = Model.fit(train_data, epochs=5, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Again Saving Model-

Model.save('bird8.h5')

In [ ]:
#Loading Model-

Model.load_weights('bird8.h5')

In [ ]:
history = Model.fit(train_data, epochs=5, batch_size=64,
                    steps_per_epoch=len(train_data),
                    callbacks=callbacks,
                    workers=1, use_multiprocessing=False,
                    validation_data=val_data, validation_steps=len(val_data))

In [ ]:
#Again Saving Model-

Model.save('bird9.h5')

In [ ]:
#Loading Model-

Model.load_weights('bird9.h5')

#### In this project I have been running epochs 3-5 five at time because the data is too much to process and single epoch is taking around 45 minutes to run. 

## Evaluating The Model-

In [ ]:
results = Model.evaluate(test_data, verbose=0)

print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
def plot_curves(history):

    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(history.history["loss"]))

    #plot loss
    plt.plot(epochs, loss, label = "training_loss")
    plt.plot(epochs, val_loss, label = "val_loss")
    plt.title("Loss")
    plt.xlabel("epochs")
    plt.legend()

    #plot accuracy
    plt.figure() 
    plt.plot(epochs, accuracy, label = "training_accuracy")
    plt.plot(epochs, val_accuracy, label = "val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("epochs")
    plt.legend()
    plt.show()
    plt.tight_layout()

In [ ]:
plot_curves(history)

## Predicting Test Set-

In [ ]:
pred = Model.predict(test_data)
pred = np.argmax(pred,axis=1)

In [ ]:
index =62

img , label = test_data[index]
label = test_data.labels[index]

print(img)
print(label)

In [ ]:
print(f"True Label: {BirdClasses[label]}")
print(f"Predicted Label: {BirdClasses[pred[index]]}")  

plt.imshow(img[0])
plt.show()

In [ ]:
#Display 10 random pictures from the dataset with their labels-

random_index = np.random.randint(0, len(test_data) - 1, 10)

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(25, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    randImg , randLabel = test_data[random_index[i]]
    randLabel = test_data.labels[random_index[i]]
    ax.imshow(randImg[0])
    if BirdClasses[randLabel] == BirdClasses[pred[random_index[i]]]:
      color = "green"
    else:
      color = "red"
    ax.set_title(f"True: {BirdClasses[randLabel]}\nPredicted: {BirdClasses[pred[random_index[i]]]}", color=color)
plt.show()
plt.tight_layout()

## Predicting Random Image-

In [ ]:
def load_and_prep_image(filename, img_shape = 224):
    
    img = tf.io.read_file(filename) #read image
    img = tf.image.decode_image(img) # decode the image to a tensor
    img = tf.image.resize(img, size = [img_shape, img_shape]) # resize the image
    img = img/255. # rescale the image
    
    return img

In [ ]:
def pred_and_plot(filename, class_names):

  #Import the target image and preprocess it
  img = load_and_prep_image(filename)

  #Make a prediction
  pred = Model.predict(tf.expand_dims(img, axis=0))

  #Get the predicted class
  pred_class = class_names[pred.argmax()]

  #Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
pred_and_plot("D:/test/BLACK FRANCOLIN/5.jpg", BirdClasses)

In [ ]:
pred_and_plot("D:/test/WOODLAND KINGFISHER/2.jpg", BirdClasses)    

In [ ]:
#Saving Model-

Model.save('bird5.h5')